In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)


vectorstore = Chroma.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer question using only the following context. If you don't know the answer just say you don't know, don't make it up:\n{context}"),
    ("human", "{question}")
])

chain = (
    {
        "context": retriever, 
        "question": RunnablePassthrough(),
    } 
    | prompt 
    | llm
)

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a place surrounded by barbed-wire entanglements, steel doors, and hidden machine-gun nests. The streets leading up to its outer barriers are patrolled by guards in black uniforms armed with jointed truncheons. Inside Victory Mansions, there is a tiny kitchen with minimal food supplies, such as a hunk of dark-colored bread. Additionally, there is a bottle of colorless liquid labeled Victory Gin, which has a sickly, oily smell. The atmosphere in Victory Mansions seems oppressive and controlled, as residents like Winston have to maintain a facade of quiet optimism when facing the telescreen.')